# 🏠 DeepRoof-2026: Multi-Task Training Notebook

Welcome to the official training environment for the **DeepRoof-2026 AI Roof Layout Engine**. 

### 🛠 Step 1: Initialize & Activate Venv
This cell will automatically find your project's `.venv`, activate it for this session, and fix any library/linking issues.

In [ ]:
import os
import sys
import subprocess
import torch
import ctypes
from pathlib import Path

# --- 1. DETECT PROJECT ROOT & VENV ---
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
venv_path = project_root / ".venv"

print(f"📂 Project Root: {project_root}")

if venv_path.exists():
    # Find site-packages in venv (handles different python versions)
    lib_dir = list(venv_path.glob("lib/python*/site-packages"))
    if lib_dir:
        venv_site = str(lib_dir[0])
        if venv_site not in sys.path:
            sys.path.insert(0, venv_site)
            print(f"✅ Activated Venv Site: {venv_site}")
    
    # Set sys.executable to venv python for subprocess calls
    venv_python = str(venv_path / "bin" / "python")
    if os.path.exists(venv_python):
        sys.executable = venv_python
        print(f"🐍 Using Venv Python: {sys.executable}")
else:
    print("⚠️ No .venv found in project root. Using default system environment.")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Added {project_root} to sys.path")

# --- 2. UNIVERSAL RECOVERY & PATCHING ---
def recover_environment():
    print("🔍 Initializing Repair Logic...")
    
    cuda_available = torch.cuda.is_available()
    print(f"🔍 CUDA Available: {cuda_available}")

    # 2a. Fix libcudart linking (For CUDA Env)
    if cuda_available:
        try:
            from mmcv.ops import point_sample
        except Exception as e:
            if "libcudart.so" in str(e):
                print("📦 Installing missing CUDA runtime libraries into venv...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
                try:
                    import nvidia.cuda_runtime
                    lib_root = Path(nvidia.cuda_runtime.__file__).parent.parent
                    so_files = list(lib_root.glob("**/libcudart.so*"))
                    if so_files:
                        ctypes.CDLL(str(so_files[0]), mode=ctypes.RTLD_GLOBAL)
                        print("✅ Explicitly loaded libcudart into process memory.")
                        # Persistence for next run
                        lib_dir = str(Path(so_files[0]).parent)
                        os.environ['LD_LIBRARY_PATH'] = lib_dir + os.pathsep + os.environ.get('LD_LIBRARY_PATH', '')
                except: pass
                return False

    # 2b. Repair mmsegmentation
    import importlib.util
    mmseg_spec = importlib.util.find_spec("mmseg")
    if mmseg_spec and mmseg_spec.origin:
        mmseg_init = Path(mmseg_spec.origin).parent / "__init__.py"
        with open(mmseg_init, 'r') as f:
            content = f.read()
        if "OVERRIDE by DeepRoof" not in content:
            print(f"🩹 Repairing mmsegmentation in venv at: {mmseg_init}")
            with open(mmseg_init, 'w') as f:
                f.write("""# Copyright (c) OpenMMLab. All rights reserved.
import mmcv
import mmengine
from mmengine.utils import digit_version
from .version import __version__, version_info
MMCV_MIN = '2.0.0rc4'
MMCV_MAX = '2.2.0'
MMENGINE_MIN = '0.7.1'
MMENGINE_MAX = '1.0.0'
mmcv_min_version = digit_version(MMCV_MIN)
mmcv_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmcv_version = digit_version(mmcv.__version__)
mmengine_min_version = digit_version(MMENGINE_MIN)
mmengine_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmengine_version = digit_version(mmengine.__version__)
__all__ = ['__version__', 'version_info', 'digit_version']\n""")
            return False

    # 2c. Ensure MMCV Binary
    try:
        from mmcv.ops import point_sample
        has_ops = True
    except: has_ops = False
    
    if cuda_available and not has_ops:
        print("🔄 Upgrading MMCV to CUDA version in venv...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "mmcv==2.2.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html"])
        return False
    elif not cuda_available and has_ops:
        print("🔄 Downgrading MMCV to CPU version in venv...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "mmcv==2.2.0"])
        return False

    print("✅ Environment is fully prepared and activated.")
    return True

if recover_environment():
    print(f"🚀 Kernel Ready | Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

## 📂 1. Dataset Preview

Visualize the **satellite imagery**, **instance masks**, and **surface normals**.

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = Path(data_root)
    if not data_path.is_absolute():
        data_path = project_root / data_root
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could not find train.txt at {train_file}. Run prepare_omnicity_v2_final.py first!")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img_path = str(data_path / 'images' / (sid + '.jpg'))
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        mask_path = str(data_path / 'masks' / (sid + '.png'))
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset("data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
from mmengine.config import Config

MODE = "fine-tune" 
CONFIG_FILE = str(project_root / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(project_root / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(project_root / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

if MODE == "scratch": cfg.load_from = None
print(f"✅ Configuration Validated. WorkDir: {WORK_DIR}")

## 🚀 3. Start Training


In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()